In [ ]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 14.4 MB/s 


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import sys
import numpy as np
import seaborn as sns
import statistics as stats
sns.set(color_codes=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#Last import allows multiple outputs from one cell
import warnings
# Initialize the random number generator
import random
random.seed(0)

#Additional libs
import matplotlib.patches as patches


In [ ]:
import os
import tensorflow as tf
from keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from keras.applications.vgg16 import VGG16, preprocess_input
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np
import tensorflow_addons as tfa

In [ ]:
# Useful Configuration/Setting

# suppress display of warnings
#warnings.filterwarnings('ignore')

# display all dataframe columns
pd.options.display.max_columns = None

# to set the limit to 3 decimals
pd.options.display.float_format = '{:.7f}'.format

# display all dataframe rows
pd.options.display.max_rows = None

#Setting to shows all entries in array displayed
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data_dir = Path('/content/drive/MyDrive/Capstone/Car Images/Car Images/Train Images')
test_data_dir = Path('/content/drive/MyDrive/Capstone/Car Images/Car Images/Test Images')

BATCH_SIZE = 32 
IMG_SIZE = 224

In [ ]:
import pandas as pd

#Lets read the Train Annotations

#Lets give our own column names as excel sheet does not have proper column names
columnNameList=['file_name','cor1','cor2','cor3','cor4','Image_Class']
trainAnno= pd.read_csv('/content/drive/MyDrive/Capstone/Annotations/Train Annotations.csv',names=columnNameList,header=0)
trainAnno.head(5)

,file_name,cor1,cor2,cor3,cor4,Image_Class
0,00001.jpg,39,116,569,375,14
1,00002.jpg,36,116,868,587,3
2,00003.jpg,85,109,601,381,91
3,00004.jpg,621,393,1484,1096,134
4,00005.jpg,14,36,133,99,106


In [ ]:
files = train_data_dir.rglob('*.jpg')
input=pd.DataFrame(files,columns =['a'])
input['a']=input['a'].astype(str)
df_train=input['a'].str.split('/',expand=True)
train=input.join(df_train)[['a',8]]
train.columns=('file_path','label')

In [ ]:
train.head()
len(train)

,file_path,label
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012


8144

In [ ]:
train.head()

,file_path,label
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012


In [ ]:
train['file_name']=train['file_path'].apply(lambda getFileName:getFileName.split('/')[9])

In [ ]:
train.head()

,file_path,label,file_name
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00482.jpg
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,05872.jpg
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,06576.jpg
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00719.jpg
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,04655.jpg


In [ ]:
trainAnno.head()

,file_name,cor1,cor2,cor3,cor4,Image_Class
0,00001.jpg,39,116,569,375,14
1,00002.jpg,36,116,868,587,3
2,00003.jpg,85,109,601,381,91
3,00004.jpg,621,393,1484,1096,134
4,00005.jpg,14,36,133,99,106


In [ ]:
# Lets add coordinate information to the above table
trainDF=pd.merge(train,trainAnno,on='file_name',how='inner')

In [ ]:
trainDF.head(5)

,file_path,label,file_name,cor1,cor2,cor3,cor4,Image_Class
0,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00482.jpg,80,91,589,429,184
1,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,05872.jpg,39,35,591,413,184
2,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,06576.jpg,64,347,629,577,184
3,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,00719.jpg,118,93,380,317,184
4,/content/drive/MyDrive/Capstone/Car Images/Car...,Suzuki SX4 Sedan 2012,04655.jpg,9,11,824,489,184


In [ ]:
from sklearn.model_selection import train_test_split
train_df,valid_df= train_test_split(trainDF, test_size=0.20, random_state=0, stratify=train['label'])

In [ ]:
train_df.shape
valid_df.shape

(6515, 8)

(1629, 8)

In [ ]:
#!pip install -qqq tensorflow_addons
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor
!pip install -U albumentations

  Cloning https://github.com/mjkvaak/ImageDataAugmentor to /tmp/pip-req-build-kd91g7mt
  Running command git clone -q https://github.com/mjkvaak/ImageDataAugmentor /tmp/pip-req-build-kd91g7mt
     |████████████████████████████████| 60.5 MB 1.2 MB/s 
     |████████████████████████████████| 631 kB 36.4 MB/s 
  Created wheel for ImageDataAugmentor: filename=ImageDataAugmentor-0.0.0-py3-none-any.whl size=29638 sha256=768d8bde29e5f28f6bd785cd4442ce76772da6ccb4110b9e2223ea1fcfe07974
  Stored in directory: /tmp/pip-ephem-wheel-cache-x66su22v/wheels/c9/bd/73/9cfa59d2393dae55bbcc30f5aa901f55fe531c66efebbc8fc3
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654017 sha256=56726b68b221e68cc544395797123114428ed3e688109ac5f202822d2bcd8fdf
  Stored in directory: /root/.cache/pip/wheels/89/72/98/3ebfdba1069a9a8eaaa7ae7265cfd67d63ef0197aaee2e5f9c
Successfully built ImageDataAugmentor imgaug
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.

In [ ]:
# Setting the correct order of labels to prevent mismatches
class_label_list=pd.read_csv('/content/drive/MyDrive/Capstone/Car _names_and_make.txt',names=['make'])

In [ ]:
type(class_label_list)

pandas.core.frame.DataFrame

In [ ]:
class_label_list.head()

,make
0,AM General Hummer SUV 2000
1,Acura RL Sedan 2012
2,Acura TL Sedan 2012
3,Acura TL Type-S 2008
4,Acura TSX Sedan 2012


In [ ]:
all_classes=list(class_label_list['make'])

In [ ]:
len(all_classes)

196

In [ ]:
import tensorflow as tf
from ImageDataAugmentor.image_data_augmentor import *
import albumentations

    
AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(p=0.5,limit=25),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=1.5, contrast_limit=0.1)
    ],p=1),
    #albumentations.HueSaturationValue(p=0.5),
    #albumentations.Affine(shear=20),
    albumentations.Cutout(num_holes=1, max_h_size=16,max_w_size = 16,p=1),
    #albumentations.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=15, p=.75),
    #albumentations.Blur(blur_limit=3),
    #albumentations.OpticalDistortion()
    #albumentations.RGBShift(p=0.5),
])

# dataloaders
train_datagen = ImageDataAugmentor(
        augment=AUGMENTATIONS,
        #validation_split=0.2,
        seed=42
        )


val_datagen = ImageDataAugmentor()#validation_split=0.2)


train_ds = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_data_dir,
        x_col="file_path",
        y_col="label",
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=all_classes

        )

val_ds = val_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=train_data_dir,
        x_col="file_path",
        y_col="label",
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=all_classes
        )

/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:691: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,


Found 6482 validated image filenames belonging to 196 classes.
Found 1621 validated image filenames belonging to 196 classes.


In [ ]:
(x,y)=train_ds.next()

In [ ]:
x.shape

(32, 224, 224, 3)

In [ ]:
y.shape

(32, 196)

In [ ]:
#train_ds.class_indices

In [ ]:
type(train_ds)

ImageDataAugmentor.dataframe_iterator.DataFrameIterator

In [ ]:
from tensorflow.keras import layers

def build_model(num_classes=196,IMG_SIZE=224):
  inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
  model = tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, input_tensor=inputs, weights="imagenet")

  # Freeze the pretrained weights
  model.trainable = False

  # Rebuild top
  x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
  x = layers.BatchNormalization()(x)

  top_dropout_rate = 0.5
  x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
  outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

  # Compile
  return tf.keras.Model(inputs, outputs, name="EfficientNet")

In [ ]:
from functools import partial
INIT_LR=0.001
MAX_LR=0.1

steps_per_epoch = len(train_ds)
clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
                                          maximal_learning_rate=MAX_LR,
                                          scale_fn=lambda x: 1/(2.**(x-1)),
                                          step_size=2 * steps_per_epoch
                                          )


optimizer = partial(SGD, momentum=0.9, nesterov=True)(clr)
optimizer = tfa.optimizers.SWA(optimizer)

model = build_model(num_classes=196,IMG_SIZE=224) 
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
epochs = 5 


hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=1)

Epoch 1/5
203/203 [==============================] - 2606s 13s/step - loss: 6.9902 - accuracy: 0.0438 - val_loss: 4.1374 - val_accuracy: 0.1536
Epoch 2/5
203/203 [==============================] - 82s 403ms/step - loss: 6.6366 - accuracy: 0.0972 - val_loss: 3.5169 - val_accuracy: 0.2153
Epoch 3/5
203/203 [==============================] - 82s 403ms/step - loss: 4.3705 - accuracy: 0.1361 - val_loss: 3.0065 - val_accuracy: 0.2825
Epoch 4/5
203/203 [==============================] - 82s 402ms/step - loss: 3.9208 - accuracy: 0.1856 - val_loss: 2.7024 - val_accuracy: 0.3590
Epoch 5/5
203/203 [==============================] - 82s 402ms/step - loss: 3.6733 - accuracy: 0.2194 - val_loss: 2.6600 - val_accuracy: 0.3603


In [ ]:
#end

In [ ]:
#STOP

In [ ]:
checkpoint_path = "./cp-{iter:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
model.save_weights(checkpoint_path.format(iter=1))

In [ ]:
model.evaluate(val_ds)

51/51 [==============================] - 17s 340ms/step - loss: 2.6600 - accuracy: 0.3603


[2.6600399017333984, 0.3602714240550995]

In [ ]:
from functools import partial

IMG_SIZE = 300
#IMG_SIZE = 128

train_ds = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_data_dir,
        x_col="file_path",
        y_col="label",
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=all_classes
        )

val_ds = val_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=train_data_dir,
        x_col="file_path",
        y_col="label",
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=all_classes
        )

model = build_model(num_classes=196,IMG_SIZE=IMG_SIZE)
#model.load_weights('./cp-0001.ckpt')

optimizer = partial(SGD, momentum=0.9, nesterov=True)(clr)
optimizer = tfa.optimizers.SWA(optimizer)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

#model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=1)

Found 6482 validated image filenames belonging to 196 classes.
Found 1621 validated image filenames belonging to 196 classes.


In [ ]:
model.save_weights(checkpoint_path.format(iter=2))

In [ ]:
for layer in model.layers[-20:]:
  if not isinstance(layer, layers.BatchNormalization):
    layer.trainable = True

INIT_LR=0.001
MAX_LR=0.1

steps_per_epoch = len(train_ds)
clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
                                          maximal_learning_rate=MAX_LR,
                                          scale_fn=lambda x: 1/(2.**(x-1)),
                                          step_size=2 * steps_per_epoch
                                          )

#model.load_weights('./cp-0002.ckpt')

model.compile(optimizer=tfa.optimizers.LAMB(), loss="categorical_crossentropy", metrics=["accuracy"])


epochs = 5  
model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=1)

Epoch 1/5
203/203 [==============================] - 102s 447ms/step - loss: 5.8607 - accuracy: 0.0234 - val_loss: 4.0894 - val_accuracy: 0.1437
Epoch 2/5
203/203 [==============================] - 88s 434ms/step - loss: 4.8313 - accuracy: 0.0713 - val_loss: 3.2636 - val_accuracy: 0.2529
Epoch 3/5
203/203 [==============================] - 89s 438ms/step - loss: 4.3086 - accuracy: 0.1134 - val_loss: 2.7951 - val_accuracy: 0.3270
Epoch 4/5
203/203 [==============================] - 89s 437ms/step - loss: 3.9361 - accuracy: 0.1689 - val_loss: 2.4848 - val_accuracy: 0.3985
Epoch 5/5
203/203 [==============================] - 89s 436ms/step - loss: 3.6731 - accuracy: 0.2151 - val_loss: 2.2582 - val_accuracy: 0.4584


In [ ]:
model.save_weights(checkpoint_path.format(iter=3))

In [ ]:
callback = EarlyStopping(monitor='val_accuracy', patience=7, min_delta=0.01)
mc = ModelCheckpoint('/content/drive/MyDrive/Capstone/Pickled_Info/classification_best_weights_300_sync.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True,save_weights_only=True)

epochs = 10
model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=1, callbacks=[mc,callback])

Epoch 1/10
203/203 [==============================] - ETA: 0s - loss: 1.9938 - accuracy: 0.5844
Epoch 1: val_accuracy improved from -inf to 0.68661, saving model to /content/drive/MyDrive/Capstone/Pickled_Info/classification_best_weights_300_sync.h5
203/203 [==============================] - 93s 458ms/step - loss: 1.9938 - accuracy: 0.5844 - val_loss: 1.1005 - val_accuracy: 0.6866
Epoch 2/10
203/203 [==============================] - ETA: 0s - loss: 1.9898 - accuracy: 0.5855
Epoch 2: val_accuracy improved from 0.68661 to 0.69278, saving model to /content/drive/MyDrive/Capstone/Pickled_Info/classification_best_weights_300_sync.h5
203/203 [==============================] - 91s 448ms/step - loss: 1.9898 - accuracy: 0.5855 - val_loss: 1.0850 - val_accuracy: 0.6928
Epoch 3/10
203/203 [==============================] - ETA: 0s - loss: 1.9678 - accuracy: 0.5943
Epoch 3: val_accuracy improved from 0.69278 to 0.69895, saving model to /content/drive/MyDrive/Capstone/Pickled_Info/classification_b

In [ ]:
model.load_weights('/content/drive/MyDrive/Capstone/Pickled_Info/classification_best_weights_300_sync.h5')

In [ ]:
model.evaluate(val_ds)

51/51 [==============================] - 16s 313ms/step - loss: 1.0867 - accuracy: 0.7045


[1.0867247581481934, 0.7045034170150757]

In [ ]:
#STOP

In [ ]:
########################################################### Pickle Validation Data###########################################################################

We will pickle Validation data set in a batch of 1600 for re-use in other notebooks

In [ ]:
IMG_SIZE = 300
BATCH_SIZE=1600

In [ ]:
from functools import partial
import tensorflow as tf
from ImageDataAugmentor.image_data_augmentor import *
import albumentations

    
AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(p=0.5,limit=25),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=1.5, contrast_limit=0.1)
    ],p=1),
    #albumentations.HueSaturationValue(p=0.5),
    #albumentations.Affine(shear=20),
    albumentations.Cutout(num_holes=1, max_h_size=16,max_w_size = 16,p=1),
    #albumentations.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=15, p=.75),
    #albumentations.Blur(blur_limit=3),
    #albumentations.OpticalDistortion()
    #albumentations.RGBShift(p=0.5),
])

# dataloaders
train_datagen = ImageDataAugmentor(
        augment=AUGMENTATIONS,
        #validation_split=0.2,
        seed=42
        )


val_datagen = ImageDataAugmentor()#validation_split=0.2)



train_ds = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_data_dir,
        x_col="file_path",
        y_col="label",
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=all_classes
        )

val_ds = val_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=train_data_dir,
        x_col="file_path",
        y_col="label",
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=all_classes
        )


/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:691: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,


Found 6482 validated image filenames belonging to 196 classes.
Found 1621 validated image filenames belonging to 196 classes.


In [ ]:
(X_Batch,Y_Batch)=val_ds.next()

In [ ]:
np.save("/content/drive/MyDrive/Capstone/Pickled_Info/X_Batcht_ClassIp.npy",X_Batch)
np.save("/content/drive/MyDrive/Capstone/Pickled_Info/Y_Batch_ClassOp.npy",Y_Batch)

In [ ]:
X_Batch.shape
Y_Batch.shape

(1600, 300, 300, 3)

(1600, 196)